In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as ticker
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
df = pd.read_excel('/content/Confectionary [4564] (1).xlsx')

In [ ]:
print(df.head())

In [ ]:
# Data Cleaning and Transformation
new_columns = {
    'Country(UK)': 'Region',
    'Cost(£)': 'Cost',
    'Profit(£)': 'Profit',
    'Revenue(£)': 'Revenue'
}
df.rename(columns=new_columns, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Confectionary'] = df['Confectionary'].replace('Choclate Chunk', 'Chocolate Chunk')

print("\n--- Data Info After Datetime Conversion and Renaming ---")
print(df.info())

In [ ]:
# Handling Missing Values (Imputation)
print("\n--- Missing Value Count (Before Imputation) ---")
print(df.isnull().sum())


missing_cost_mask = df['Cost'].isnull() & df['Revenue'].notnull() & df['Profit'].notnull()
df.loc[missing_cost_mask, 'Cost'] = df.loc[missing_cost_mask, 'Revenue'] - df.loc[missing_cost_mask, 'Profit']


missing_profit_mask = df['Profit'].isnull() & df['Revenue'].notnull() & df['Cost'].notnull()
df.loc[missing_profit_mask, 'Profit'] = df.loc[missing_profit_mask, 'Revenue'] - df.loc[missing_profit_mask, 'Cost']

print("\n--- Missing Value Count (After Imputation Logic) ---")
print(df.isnull().sum())

In [ ]:
# Data Quality Assessment (Final Cleanup)
df_clean = df[(df['Profit'] > 0) & (df['Revenue'] > 0)].copy()

df_clean.dropna(subset=['Units Sold'], inplace=True)

print("\n--- Final Cleaned Data Info ---")
print(df_clean.info())
print("\n--- Final Cleaned Data Sample ---")
print(df_clean.head())

In [ ]:
# --- 3.1 Regional Distribution of Sales and Profit ---
regional_summary = df_clean.groupby('Region')[['Revenue', 'Profit']].sum().reset_index()
regional_summary['Profit Margin (%)'] = (regional_summary['Profit'] / regional_summary['Revenue']) * 100
regional_summary.sort_values(by='Revenue', ascending=False, inplace=True)

print("\n--- 3.1 Regional Sales and Profit Summary ---")

regional_summary.to_csv("Regional_Summary.csv", index=False)

# Plotting Regional Revenue and Profit
fig, ax = plt.subplots(figsize=(10, 6))
regional_summary_melt = regional_summary.melt(id_vars='Region', value_vars=['Revenue', 'Profit'],
                                              var_name='Metric', value_name='Value (£)')
sns.barplot(x='Region', y='Value (£)', hue='Metric', data=regional_summary_melt, ax=ax, palette={'Revenue': 'cornflowerblue', 'Profit': 'darkorange'})
ax.set_title('Figure 1: Total Revenue and Profit by UK Region', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Region', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Regional_Sales_Profit.png')
plt.close()
regional_summary

In [ ]:
# --- 3.2 Comparative Overview of Confectionary Types ---
confectionary_summary = df_clean.groupby('Confectionary')[['Revenue', 'Profit']].sum().reset_index()
confectionary_summary['Profit Margin (%)'] = (confectionary_summary['Profit'] / confectionary_summary['Revenue']) * 100
confectionary_summary.sort_values(by='Profit Margin (%)', ascending=False, inplace=True)

print("\n--- 3.2 Confectionary Type Sales and Profit Summary (Sorted by Margin) ---")
confectionary_summary
confectionary_summary.to_csv("Confectionary_Summary.csv", index=False)

# Plotting Confectionary Revenue and Profit (sorted by Revenue for visual clarity)
fig, ax = plt.subplots(figsize=(10, 6))
confectionary_summary_melt = confectionary_summary.melt(id_vars='Confectionary', value_vars=['Revenue', 'Profit'],
                                                      var_name='Metric', value_name='Value (£)')
confectionary_order = confectionary_summary.sort_values(by='Revenue', ascending=False)['Confectionary'].tolist()

sns.barplot(x='Confectionary', y='Value (£)', hue='Metric', data=confectionary_summary_melt, ax=ax, palette={'Revenue': 'seagreen', 'Profit': 'goldenrod'}, order=confectionary_order)
ax.set_title('Figure 2: Total Revenue and Profit by Confectionery Type', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Confectionery Type', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Confectionary_Sales_Profit.png')
plt.close()
confectionary_summary

In [ ]:
# --- 3.3 Time Series Analysis ---
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month

# Regional Peak Sales Analysis
regional_yearly_sales = df_clean.groupby(['Year', 'Region'])['Revenue'].sum().reset_index()
peak_sale = regional_yearly_sales.loc[regional_yearly_sales['Revenue'].idxmax()]
time_series_summary = df_clean.groupby('Year')['Revenue'].sum().reset_index()

print("\n--- 3.3 Peak Regional Sales (Region with the single highest yearly revenue) ---")
print(peak_sale)

# Plotting Regional Yearly Revenue to show trends and peak
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Revenue', hue='Region', data=regional_yearly_sales, marker='o')
plt.title('Figure 3: Yearly Revenue Trend by Region (2000-2005)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('Regional_Yearly_Revenue.png')
plt.close()
time_series_summary

In [ ]:
# --- 3.4 Correlation Analysis between Sales, Revenue, and Profit Margin ---
df_clean['Profit_Margin'] = df_clean['Profit'] / df_clean['Revenue']
correlation_data = df_clean[['Units Sold', 'Revenue', 'Profit', 'Profit_Margin']]
correlation_matrix = correlation_data.corr()

print("\n--- 3.4 Correlation Matrix ---")
print(correlation_matrix)
correlation_matrix.to_csv("Correlation_Matrix.csv")

# Plotting the Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".2f",
            linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Figure 4: Correlation Analysis of Key Financial Metrics', fontsize=14)
plt.tight_layout()
plt.savefig('Correlation_Heatmap.png')
plt.close()

In [ ]:
# 1. Aggregate Revenue, Profit, and calculate Margin by Region and Confectionery
regional_product_summary = df_clean.groupby(['Region', 'Confectionary'])[['Revenue', 'Profit']].sum().reset_index()
regional_product_summary['Profit Margin (%)'] = (regional_product_summary['Profit'] / regional_product_summary['Revenue']) * 100

print("\n--- 4.1 Confectionery Performance Metrics (by Region) ---")
print(regional_product_summary)
regional_product_summary.to_csv("Regional_Product_Metrics.csv", index=False)

In [ ]:
# 2. Find the index corresponding to the maximum Profit Margin for each Region
idx_max = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmax()
largest_margins = regional_product_summary.loc[idx_max].sort_values(by='Profit Margin (%)', ascending=False).reset_index(drop=True)

print("\n--- 4.2 Identification of Confectionery with Largest Profit Margins (by Region) ---")
print(largest_margins)
largest_margins.to_csv("Largest_Profit_Margins_by_Region.csv", index=False)

In [ ]:
# 3. Find the index corresponding to the minimum Profit Margin for each Region
idx_min = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmin()
smallest_margins = regional_product_summary.loc[idx_min].sort_values(by='Profit Margin (%)', ascending=True).reset_index(drop=True)

print("\n--- 4.3 Identification of Confectionery with Smallest Profit Margins (by Region) ---")
smallest_margins

In [ ]:
# Figure 5: Boxplot of Units Sold Distribution by Confectionery Type
# Shows volatility and average order size (Logistics Insight)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Confectionary', y='Units Sold', data=df_clean, palette='pastel')
plt.title('Figure 5: Distribution of Units Sold by Confectionery Type', fontsize=14)
plt.xlabel('Confectionery Type', fontsize=12)
plt.ylabel('Units Sold (Log Scale)', fontsize=12)
plt.yscale('log') # Use log scale for better visibility of outliers and distribution
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('Boxplot_Units_Sold.png')
plt.close()

In [ ]:
# Figure 6: Scatter Plot of Profit vs. Revenue by Region
# Shows the Volume-Margin Trade-Off (Pricing Insight)
plt.figure(figsize=(12, 7))
sns.scatterplot(x='Revenue', y='Profit', hue='Region', data=df_clean, style='Region', s=100, alpha=0.6)

# Add a reference line to visualize non-linearity
x_max = df_clean['Revenue'].max()
plt.plot([0, x_max], [0, df_clean['Profit'].max()], color='red', linestyle='--', label='Max Observed Profit Line')

plt.title('Figure 6: Transactional Profit vs. Revenue by Region', fontsize=14)
plt.xlabel('Revenue (£)', fontsize=12)
plt.ylabel('Profit (£)', fontsize=12)
plt.gca().ticklabel_format(style='plain', axis='both')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Region', loc='upper left')
plt.grid(axis='both', linestyle='--')
plt.tight_layout()
plt.savefig('Scatter_Profit_Revenue.png')
plt.close()

In [ ]:
# --- 5.2 Interactive Visualisations (Plotly for Dashboard Component) ---

# Aggregate data for the interactive visualization (Revenue by Region and Confectionery)
regional_product_rev = df_clean.groupby(['Region', 'Confectionary'])['Revenue'].sum().reset_index()
regional_product_rev.rename(columns={'Revenue': 'Total Revenue (£)'}, inplace=True)

# Plotly Bar Chart: Total Revenue by Confectionery, allowing color/filter by Region
fig_plotly = px.bar(
    regional_product_rev,
    x='Confectionary',
    y='Total Revenue (£)',
    color='Region',
    title='Interactive Component: Total Revenue by Confectionery (Filterable by Region)',
    labels={'Total Revenue (£)': 'Total Revenue (£)'},
    height=500
)

# Add layout for better large number display and interactivity features
fig_plotly.update_layout(
    yaxis_tickformat='.2s',  # Scientific notation for large numbers
    hovermode="x unified"
)

# Save Plotly figure to a JSON file (required dashboard output)
plotly_json = fig_plotly.to_json()
with open('Interactive_Revenue_Plot.json', 'w') as f:
    f.write(plotly_json)

print("Interactive Plotly JSON file 'Interactive_Revenue_Plot.json' created for dashboard assembly.")

In [ ]:
# --- 2. Dashboard Setup ---

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define options for selectors
all_regions = sorted(df_clean['Region'].unique().tolist())
all_confectioneries = sorted(df_clean['Confectionary'].unique().tolist())
min_year = df_clean['Year'].min()
max_year = df_clean['Year'].max()

# --- 3. Dashboard Layout (6.5 Mockup) ---

sidebar = dbc.Card(
    [
        html.H4("Profit-First Explorer Filters", className="card-title"),
        html.Hr(),

        # Regional Selector (6.3)
        html.Div(
            [
                dbc.Label("Select Region(s)"),
                dcc.Checklist(
                    id='region-selector',
                    options=[{'label': region, 'value': region} for region in all_regions],
                    value=all_regions, # Default to all selected
                    inline=False,
                    className="mb-3",
                    labelStyle={'display': 'block'}
                ),
            ]
        ),

        # Confectionary Type Selector (6.3)
        html.Div(
            [
                dbc.Label("Confectionery Type"),
                dcc.Dropdown(
                    id='confectionery-selector',
                    options=[{'label': 'All Types', 'value': 'All Types'}] +
                            [{'label': type, 'value': type} for type in all_confectioneries],
                    value='All Types',
                    clearable=False,
                    className="mb-3"
                ),
            ]
        ),

        # Time Range Slider (6.3)
        html.Div(
            [
                dbc.Label("Time Range Slider (Year)"),
                dcc.RangeSlider(
                    id='year-slider',
                    min=min_year,
                    max=max_year,
                    value=[min_year, max_year],
                    marks={str(year): str(year) for year in range(min_year, max_year + 1, 1)},
                    step=1
                ),
                html.Br()
            ]
        )
    ],
    body=True,
    className="h-100",
)

content = dbc.Container([
    html.H1("BCC Profitability Dashboard: The Distribution Strategy", className="my-4 text-center"),

    # KPI Cards Row
    dbc.Row([
        dbc.Col(dbc.Card(html.H2(id='kpi-revenue', className="text-center"), body=True, color="primary", inverse=True), md=6),
        dbc.Col(dbc.Card(html.H2(id='kpi-margin', className="text-center"), body=True, color="success", inverse=True), md=6),
    ], className="mb-4"),

    # Profit vs. Revenue Graph (6.3)
    dbc.Row([
        dbc.Col(dcc.Graph(id='profit-revenue-graph'), md=12)
    ]),

], fluid=True)


app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, md=3, className="p-3"),
                dbc.Col(content, md=9, className="p-3"),
            ],
            className="h-100"
        )
    ],
    fluid=True,
    className="vh-100"
)

# --- 4. Interactivity (Callbacks: 6.4) ---

@app.callback(
    [Output('profit-revenue-graph', 'figure'),
     Output('kpi-revenue', 'children'),
     Output('kpi-margin', 'children')],
    [Input('region-selector', 'value'),
     Input('confectionery-selector', 'value'),
     Input('year-slider', 'value')]
)
def update_dashboard(selected_regions, selected_confectionery, year_range):
    # 1. Filter Data based on user inputs
    dff = df_clean[df_clean['Region'].isin(selected_regions)]

    if selected_confectionery != 'All Types':
        dff = dff[dff['Confectionary'] == selected_confectionery]

    dff = dff[(dff['Year'] >= year_range[0]) & (dff['Year'] <= year_range[1])]

    # 2. KPI Calculation
    total_revenue = dff['Revenue'].sum()
    total_profit = dff['Profit'].sum()
    average_margin = (total_profit / total_revenue) if total_revenue > 0 else 0

    # 3. Graph Generation (Profit vs. Revenue Scatter Plot)
    fig = px.scatter(
        dff,
        x='Revenue',
        y='Profit',
        color='Region',
        size='Units Sold', # Visualizing the units sold through size
        hover_data=['Confectionary', 'Date'],
        title="Profit vs. Revenue by Region and Confectionary Type (Filtered Data)",
        labels={'Revenue': 'Total Revenue (£)', 'Profit': 'Total Profit (£)'}
    )

    # Add a visual reference for overall Profit Margin (as a line from origin)
    if total_revenue > 0:
        fig.add_shape(
            type='line',
            x0=0, y0=0,
            x1=total_revenue, y1=total_profit,
            line=dict(color="Red", width=2, dash="dash"),
            name="Avg Margin Line"
        )

    fig.update_layout(
        xaxis_tickformat='.2s',
        yaxis_tickformat='.2s',
        transition_duration=500
    )

    # 4. Return KPIs
    kpi_rev = f"Total Revenue: £{total_revenue:,.0f}"
    kpi_mar = f"Avg Margin: {average_margin:.2%}"

    return fig, kpi_rev, kpi_mar


if __name__ == '__main__':
    print(f"Starting Dash app. Go to http://127.0.0.1:8050/")
    # Debug=True is used here for demonstration/development purposes
    app.run(debug=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as ticker
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
df = pd.read_excel('/content/Confectionary [4564] (1).xlsx')

In [ ]:
print(df.head())

In [ ]:
# Data Cleaning and Transformation
new_columns = {
    'Country(UK)': 'Region',
    'Cost(£)': 'Cost',
    'Profit(£)': 'Profit',
    'Revenue(£)': 'Revenue'
}
df.rename(columns=new_columns, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Confectionary'] = df['Confectionary'].replace('Choclate Chunk', 'Chocolate Chunk')

print("\n--- Data Info After Datetime Conversion and Renaming ---")
print(df.info())

In [ ]:
# Handling Missing Values (Imputation)
print("\n--- Missing Value Count (Before Imputation) ---")
print(df.isnull().sum())


missing_cost_mask = df['Cost'].isnull() & df['Revenue'].notnull() & df['Profit'].notnull()
df.loc[missing_cost_mask, 'Cost'] = df.loc[missing_cost_mask, 'Revenue'] - df.loc[missing_cost_mask, 'Profit']


missing_profit_mask = df['Profit'].isnull() & df['Revenue'].notnull() & df['Cost'].notnull()
df.loc[missing_profit_mask, 'Profit'] = df.loc[missing_profit_mask, 'Revenue'] - df.loc[missing_profit_mask, 'Cost']

print("\n--- Missing Value Count (After Imputation Logic) ---")
print(df.isnull().sum())

In [ ]:
# Data Quality Assessment (Final Cleanup)
df_clean = df[(df['Profit'] > 0) & (df['Revenue'] > 0)].copy()

df_clean.dropna(subset=['Units Sold'], inplace=True)

print("\n--- Final Cleaned Data Info ---")
print(df_clean.info())
print("\n--- Final Cleaned Data Sample ---")
print(df_clean.head())

In [ ]:
# --- 3.1 Regional Distribution of Sales and Profit ---
regional_summary = df_clean.groupby('Region')[['Revenue', 'Profit']].sum().reset_index()
regional_summary['Profit Margin (%)'] = (regional_summary['Profit'] / regional_summary['Revenue']) * 100
regional_summary.sort_values(by='Revenue', ascending=False, inplace=True)

print("\n--- 3.1 Regional Sales and Profit Summary ---")

regional_summary.to_csv("Regional_Summary.csv", index=False)

# Plotting Regional Revenue and Profit
fig, ax = plt.subplots(figsize=(10, 6))
regional_summary_melt = regional_summary.melt(id_vars='Region', value_vars=['Revenue', 'Profit'],
                                              var_name='Metric', value_name='Value (£)')
sns.barplot(x='Region', y='Value (£)', hue='Metric', data=regional_summary_melt, ax=ax, palette={'Revenue': 'cornflowerblue', 'Profit': 'darkorange'})
ax.set_title('Figure 1: Total Revenue and Profit by UK Region', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Region', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Regional_Sales_Profit.png')
plt.close()
regional_summary

In [ ]:
# --- 3.2 Comparative Overview of Confectionary Types ---
confectionary_summary = df_clean.groupby('Confectionary')[['Revenue', 'Profit']].sum().reset_index()
confectionary_summary['Profit Margin (%)'] = (confectionary_summary['Profit'] / confectionary_summary['Revenue']) * 100
confectionary_summary.sort_values(by='Profit Margin (%)', ascending=False, inplace=True)

print("\n--- 3.2 Confectionary Type Sales and Profit Summary (Sorted by Margin) ---")
confectionary_summary
confectionary_summary.to_csv("Confectionary_Summary.csv", index=False)

# Plotting Confectionary Revenue and Profit (sorted by Revenue for visual clarity)
fig, ax = plt.subplots(figsize=(10, 6))
confectionary_summary_melt = confectionary_summary.melt(id_vars='Confectionary', value_vars=['Revenue', 'Profit'],
                                                      var_name='Metric', value_name='Value (£)')
confectionary_order = confectionary_summary.sort_values(by='Revenue', ascending=False)['Confectionary'].tolist()

sns.barplot(x='Confectionary', y='Value (£)', hue='Metric', data=confectionary_summary_melt, ax=ax, palette={'Revenue': 'seagreen', 'Profit': 'goldenrod'}, order=confectionary_order)
ax.set_title('Figure 2: Total Revenue and Profit by Confectionery Type', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Confectionery Type', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Confectionary_Sales_Profit.png')
plt.close()
confectionary_summary

In [ ]:
# --- 3.3 Time Series Analysis ---
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month

# Regional Peak Sales Analysis
regional_yearly_sales = df_clean.groupby(['Year', 'Region'])['Revenue'].sum().reset_index()
peak_sale = regional_yearly_sales.loc[regional_yearly_sales['Revenue'].idxmax()]
time_series_summary = df_clean.groupby('Year')['Revenue'].sum().reset_index()

print("\n--- 3.3 Peak Regional Sales (Region with the single highest yearly revenue) ---")
print(peak_sale)

# Plotting Regional Yearly Revenue to show trends and peak
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Revenue', hue='Region', data=regional_yearly_sales, marker='o')
plt.title('Figure 3: Yearly Revenue Trend by Region (2000-2005)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('Regional_Yearly_Revenue.png')
plt.close()
time_series_summary

In [ ]:
# --- 3.4 Correlation Analysis between Sales, Revenue, and Profit Margin ---
df_clean['Profit_Margin'] = df_clean['Profit'] / df_clean['Revenue']
correlation_data = df_clean[['Units Sold', 'Revenue', 'Profit', 'Profit_Margin']]
correlation_matrix = correlation_data.corr()

print("\n--- 3.4 Correlation Matrix ---")
print(correlation_matrix)
correlation_matrix.to_csv("Correlation_Matrix.csv")

# Plotting the Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".2f",
            linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Figure 4: Correlation Analysis of Key Financial Metrics', fontsize=14)
plt.tight_layout()
plt.savefig('Correlation_Heatmap.png')
plt.close()

In [ ]:
# 1. Aggregate Revenue, Profit, and calculate Margin by Region and Confectionery
regional_product_summary = df_clean.groupby(['Region', 'Confectionary'])[['Revenue', 'Profit']].sum().reset_index()
regional_product_summary['Profit Margin (%)'] = (regional_product_summary['Profit'] / regional_product_summary['Revenue']) * 100

print("\n--- 4.1 Confectionery Performance Metrics (by Region) ---")
print(regional_product_summary)
regional_product_summary.to_csv("Regional_Product_Metrics.csv", index=False)

In [ ]:
# 2. Find the index corresponding to the maximum Profit Margin for each Region
idx_max = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmax()
largest_margins = regional_product_summary.loc[idx_max].sort_values(by='Profit Margin (%)', ascending=False).reset_index(drop=True)

print("\n--- 4.2 Identification of Confectionery with Largest Profit Margins (by Region) ---")
print(largest_margins)
largest_margins.to_csv("Largest_Profit_Margins_by_Region.csv", index=False)

In [ ]:
# 3. Find the index corresponding to the minimum Profit Margin for each Region
idx_min = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmin()
smallest_margins = regional_product_summary.loc[idx_min].sort_values(by='Profit Margin (%)', ascending=True).reset_index(drop=True)

print("\n--- 4.3 Identification of Confectionery with Smallest Profit Margins (by Region) ---")
smallest_margins

In [ ]:
# Figure 5: Boxplot of Units Sold Distribution by Confectionery Type
# Shows volatility and average order size (Logistics Insight)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Confectionary', y='Units Sold', data=df_clean, palette='pastel')
plt.title('Figure 5: Distribution of Units Sold by Confectionery Type', fontsize=14)
plt.xlabel('Confectionery Type', fontsize=12)
plt.ylabel('Units Sold (Log Scale)', fontsize=12)
plt.yscale('log') # Use log scale for better visibility of outliers and distribution
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('Boxplot_Units_Sold.png')
plt.close()

In [ ]:
# Figure 6: Scatter Plot of Profit vs. Revenue by Region
# Shows the Volume-Margin Trade-Off (Pricing Insight)
plt.figure(figsize=(12, 7))
sns.scatterplot(x='Revenue', y='Profit', hue='Region', data=df_clean, style='Region', s=100, alpha=0.6)

# Add a reference line to visualize non-linearity
x_max = df_clean['Revenue'].max()
plt.plot([0, x_max], [0, df_clean['Profit'].max()], color='red', linestyle='--', label='Max Observed Profit Line')

plt.title('Figure 6: Transactional Profit vs. Revenue by Region', fontsize=14)
plt.xlabel('Revenue (£)', fontsize=12)
plt.ylabel('Profit (£)', fontsize=12)
plt.gca().ticklabel_format(style='plain', axis='both')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Region', loc='upper left')
plt.grid(axis='both', linestyle='--')
plt.tight_layout()
plt.savefig('Scatter_Profit_Revenue.png')
plt.close()

In [ ]:
# --- 5.2 Interactive Visualisations (Plotly for Dashboard Component) ---

# Aggregate data for the interactive visualization (Revenue by Region and Confectionery)
regional_product_rev = df_clean.groupby(['Region', 'Confectionary'])['Revenue'].sum().reset_index()
regional_product_rev.rename(columns={'Revenue': 'Total Revenue (£)'}, inplace=True)

# Plotly Bar Chart: Total Revenue by Confectionery, allowing color/filter by Region
fig_plotly = px.bar(
    regional_product_rev,
    x='Confectionary',
    y='Total Revenue (£)',
    color='Region',
    title='Interactive Component: Total Revenue by Confectionery (Filterable by Region)',
    labels={'Total Revenue (£)': 'Total Revenue (£)'},
    height=500
)

# Add layout for better large number display and interactivity features
fig_plotly.update_layout(
    yaxis_tickformat='.2s',  # Scientific notation for large numbers
    hovermode="x unified"
)

# Save Plotly figure to a JSON file (required dashboard output)
plotly_json = fig_plotly.to_json()
with open('Interactive_Revenue_Plot.json', 'w') as f:
    f.write(plotly_json)

print("Interactive Plotly JSON file 'Interactive_Revenue_Plot.json' created for dashboard assembly.")

In [ ]:
# --- 2. Dashboard Setup ---

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define options for selectors
all_regions = sorted(df_clean['Region'].unique().tolist())
all_confectioneries = sorted(df_clean['Confectionary'].unique().tolist())
min_year = df_clean['Year'].min()
max_year = df_clean['Year'].max()

# --- 3. Dashboard Layout (6.5 Mockup) ---

sidebar = dbc.Card(
    [
        html.H4("Profit-First Explorer Filters", className="card-title"),
        html.Hr(),

        # Regional Selector (6.3)
        html.Div(
            [
                dbc.Label("Select Region(s)"),
                dcc.Checklist(
                    id='region-selector',
                    options=[{'label': region, 'value': region} for region in all_regions],
                    value=all_regions, # Default to all selected
                    inline=False,
                    className="mb-3",
                    labelStyle={'display': 'block'}
                ),
            ]
        ),

        # Confectionary Type Selector (6.3)
        html.Div(
            [
                dbc.Label("Confectionery Type"),
                dcc.Dropdown(
                    id='confectionery-selector',
                    options=[{'label': 'All Types', 'value': 'All Types'}] +
                            [{'label': type, 'value': type} for type in all_confectioneries],
                    value='All Types',
                    clearable=False,
                    className="mb-3"
                ),
            ]
        ),

        # Time Range Slider (6.3)
        html.Div(
            [
                dbc.Label("Time Range Slider (Year)"),
                dcc.RangeSlider(
                    id='year-slider',
                    min=min_year,
                    max=max_year,
                    value=[min_year, max_year],
                    marks={str(year): str(year) for year in range(min_year, max_year + 1, 1)},
                    step=1
                ),
                html.Br()
            ]
        )
    ],
    body=True,
    className="h-100",
)

content = dbc.Container([
    html.H1("BCC Profitability Dashboard: The Distribution Strategy", className="my-4 text-center"),

    # KPI Cards Row
    dbc.Row([
        dbc.Col(dbc.Card(html.H2(id='kpi-revenue', className="text-center"), body=True, color="primary", inverse=True), md=6),
        dbc.Col(dbc.Card(html.H2(id='kpi-margin', className="text-center"), body=True, color="success", inverse=True), md=6),
    ], className="mb-4"),

    # Profit vs. Revenue Graph (6.3)
    dbc.Row([
        dbc.Col(dcc.Graph(id='profit-revenue-graph'), md=12)
    ]),

], fluid=True)


app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, md=3, className="p-3"),
                dbc.Col(content, md=9, className="p-3"),
            ],
            className="h-100"
        )
    ],
    fluid=True,
    className="vh-100"
)

# --- 4. Interactivity (Callbacks: 6.4) ---

@app.callback(
    [Output('profit-revenue-graph', 'figure'),
     Output('kpi-revenue', 'children'),
     Output('kpi-margin', 'children')],
    [Input('region-selector', 'value'),
     Input('confectionery-selector', 'value'),
     Input('year-slider', 'value')]
)
def update_dashboard(selected_regions, selected_confectionery, year_range):
    # 1. Filter Data based on user inputs
    dff = df_clean[df_clean['Region'].isin(selected_regions)]

    if selected_confectionery != 'All Types':
        dff = dff[dff['Confectionary'] == selected_confectionery]

    dff = dff[(dff['Year'] >= year_range[0]) & (dff['Year'] <= year_range[1])]

    # 2. KPI Calculation
    total_revenue = dff['Revenue'].sum()
    total_profit = dff['Profit'].sum()
    average_margin = (total_profit / total_revenue) if total_revenue > 0 else 0

    # 3. Graph Generation (Profit vs. Revenue Scatter Plot)
    fig = px.scatter(
        dff,
        x='Revenue',
        y='Profit',
        color='Region',
        size='Units Sold', # Visualizing the units sold through size
        hover_data=['Confectionary', 'Date'],
        title="Profit vs. Revenue by Region and Confectionary Type (Filtered Data)",
        labels={'Revenue': 'Total Revenue (£)', 'Profit': 'Total Profit (£)'}
    )

    # Add a visual reference for overall Profit Margin (as a line from origin)
    if total_revenue > 0:
        fig.add_shape(
            type='line',
            x0=0, y0=0,
            x1=total_revenue, y1=total_profit,
            line=dict(color="Red", width=2, dash="dash"),
            name="Avg Margin Line"
        )

    fig.update_layout(
        xaxis_tickformat='.2s',
        yaxis_tickformat='.2s',
        transition_duration=500
    )

    # 4. Return KPIs
    kpi_rev = f"Total Revenue: £{total_revenue:,.0f}"
    kpi_mar = f"Avg Margin: {average_margin:.2%}"

    return fig, kpi_rev, kpi_mar


if __name__ == '__main__':
    print(f"Starting Dash app. Go to http://127.0.0.1:8050/")
    # Debug=True is used here for demonstration/development purposes
    app.run(debug=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as ticker
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
df = pd.read_excel('/content/Confectionary [4564] (1).xlsx')

In [ ]:
print(df.head())

In [ ]:
# Data Cleaning and Transformation
new_columns = {
    'Country(UK)': 'Region',
    'Cost(£)': 'Cost',
    'Profit(£)': 'Profit',
    'Revenue(£)': 'Revenue'
}
df.rename(columns=new_columns, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Confectionary'] = df['Confectionary'].replace('Choclate Chunk', 'Chocolate Chunk')

print("\n--- Data Info After Datetime Conversion and Renaming ---")
print(df.info())

In [ ]:
# Handling Missing Values (Imputation)
print("\n--- Missing Value Count (Before Imputation) ---")
print(df.isnull().sum())


missing_cost_mask = df['Cost'].isnull() & df['Revenue'].notnull() & df['Profit'].notnull()
df.loc[missing_cost_mask, 'Cost'] = df.loc[missing_cost_mask, 'Revenue'] - df.loc[missing_cost_mask, 'Profit']


missing_profit_mask = df['Profit'].isnull() & df['Revenue'].notnull() & df['Cost'].notnull()
df.loc[missing_profit_mask, 'Profit'] = df.loc[missing_profit_mask, 'Revenue'] - df.loc[missing_profit_mask, 'Cost']

print("\n--- Missing Value Count (After Imputation Logic) ---")
print(df.isnull().sum())

In [ ]:
# Data Quality Assessment (Final Cleanup)
df_clean = df[(df['Profit'] > 0) & (df['Revenue'] > 0)].copy()

df_clean.dropna(subset=['Units Sold'], inplace=True)

print("\n--- Final Cleaned Data Info ---")
print(df_clean.info())
print("\n--- Final Cleaned Data Sample ---")
print(df_clean.head())

In [ ]:
# --- 3.1 Regional Distribution of Sales and Profit ---
regional_summary = df_clean.groupby('Region')[['Revenue', 'Profit']].sum().reset_index()
regional_summary['Profit Margin (%)'] = (regional_summary['Profit'] / regional_summary['Revenue']) * 100
regional_summary.sort_values(by='Revenue', ascending=False, inplace=True)

print("\n--- 3.1 Regional Sales and Profit Summary ---")

regional_summary.to_csv("Regional_Summary.csv", index=False)

# Plotting Regional Revenue and Profit
fig, ax = plt.subplots(figsize=(10, 6))
regional_summary_melt = regional_summary.melt(id_vars='Region', value_vars=['Revenue', 'Profit'],
                                              var_name='Metric', value_name='Value (£)')
sns.barplot(x='Region', y='Value (£)', hue='Metric', data=regional_summary_melt, ax=ax, palette={'Revenue': 'cornflowerblue', 'Profit': 'darkorange'})
ax.set_title('Figure 1: Total Revenue and Profit by UK Region', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Region', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Regional_Sales_Profit.png')
plt.close()
regional_summary

In [ ]:
# --- 3.2 Comparative Overview of Confectionary Types ---
confectionary_summary = df_clean.groupby('Confectionary')[['Revenue', 'Profit']].sum().reset_index()
confectionary_summary['Profit Margin (%)'] = (confectionary_summary['Profit'] / confectionary_summary['Revenue']) * 100
confectionary_summary.sort_values(by='Profit Margin (%)', ascending=False, inplace=True)

print("\n--- 3.2 Confectionary Type Sales and Profit Summary (Sorted by Margin) ---")
confectionary_summary
confectionary_summary.to_csv("Confectionary_Summary.csv", index=False)

# Plotting Confectionary Revenue and Profit (sorted by Revenue for visual clarity)
fig, ax = plt.subplots(figsize=(10, 6))
confectionary_summary_melt = confectionary_summary.melt(id_vars='Confectionary', value_vars=['Revenue', 'Profit'],
                                                      var_name='Metric', value_name='Value (£)')
confectionary_order = confectionary_summary.sort_values(by='Revenue', ascending=False)['Confectionary'].tolist()

sns.barplot(x='Confectionary', y='Value (£)', hue='Metric', data=confectionary_summary_melt, ax=ax, palette={'Revenue': 'seagreen', 'Profit': 'goldenrod'}, order=confectionary_order)
ax.set_title('Figure 2: Total Revenue and Profit by Confectionery Type', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Confectionery Type', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Confectionary_Sales_Profit.png')
plt.close()
confectionary_summary

In [ ]:
# --- 3.3 Time Series Analysis ---
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month

# Regional Peak Sales Analysis
regional_yearly_sales = df_clean.groupby(['Year', 'Region'])['Revenue'].sum().reset_index()
peak_sale = regional_yearly_sales.loc[regional_yearly_sales['Revenue'].idxmax()]
time_series_summary = df_clean.groupby('Year')['Revenue'].sum().reset_index()

print("\n--- 3.3 Peak Regional Sales (Region with the single highest yearly revenue) ---")
print(peak_sale)

# Plotting Regional Yearly Revenue to show trends and peak
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Revenue', hue='Region', data=regional_yearly_sales, marker='o')
plt.title('Figure 3: Yearly Revenue Trend by Region (2000-2005)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('Regional_Yearly_Revenue.png')
plt.close()
time_series_summary

In [ ]:
# --- 3.4 Correlation Analysis between Sales, Revenue, and Profit Margin ---
df_clean['Profit_Margin'] = df_clean['Profit'] / df_clean['Revenue']
correlation_data = df_clean[['Units Sold', 'Revenue', 'Profit', 'Profit_Margin']]
correlation_matrix = correlation_data.corr()

print("\n--- 3.4 Correlation Matrix ---")
print(correlation_matrix)
correlation_matrix.to_csv("Correlation_Matrix.csv")

# Plotting the Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".2f",
            linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Figure 4: Correlation Analysis of Key Financial Metrics', fontsize=14)
plt.tight_layout()
plt.savefig('Correlation_Heatmap.png')
plt.close()

In [ ]:
# 1. Aggregate Revenue, Profit, and calculate Margin by Region and Confectionery
regional_product_summary = df_clean.groupby(['Region', 'Confectionary'])[['Revenue', 'Profit']].sum().reset_index()
regional_product_summary['Profit Margin (%)'] = (regional_product_summary['Profit'] / regional_product_summary['Revenue']) * 100

print("\n--- 4.1 Confectionery Performance Metrics (by Region) ---")
print(regional_product_summary)
regional_product_summary.to_csv("Regional_Product_Metrics.csv", index=False)

In [ ]:
# 2. Find the index corresponding to the maximum Profit Margin for each Region
idx_max = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmax()
largest_margins = regional_product_summary.loc[idx_max].sort_values(by='Profit Margin (%)', ascending=False).reset_index(drop=True)

print("\n--- 4.2 Identification of Confectionery with Largest Profit Margins (by Region) ---")
print(largest_margins)
largest_margins.to_csv("Largest_Profit_Margins_by_Region.csv", index=False)

In [ ]:
# 3. Find the index corresponding to the minimum Profit Margin for each Region
idx_min = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmin()
smallest_margins = regional_product_summary.loc[idx_min].sort_values(by='Profit Margin (%)', ascending=True).reset_index(drop=True)

print("\n--- 4.3 Identification of Confectionery with Smallest Profit Margins (by Region) ---")
smallest_margins

In [ ]:
# Figure 5: Boxplot of Units Sold Distribution by Confectionery Type
# Shows volatility and average order size (Logistics Insight)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Confectionary', y='Units Sold', data=df_clean, palette='pastel')
plt.title('Figure 5: Distribution of Units Sold by Confectionery Type', fontsize=14)
plt.xlabel('Confectionery Type', fontsize=12)
plt.ylabel('Units Sold (Log Scale)', fontsize=12)
plt.yscale('log') # Use log scale for better visibility of outliers and distribution
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('Boxplot_Units_Sold.png')
plt.close()

In [ ]:
# Figure 6: Scatter Plot of Profit vs. Revenue by Region
# Shows the Volume-Margin Trade-Off (Pricing Insight)
plt.figure(figsize=(12, 7))
sns.scatterplot(x='Revenue', y='Profit', hue='Region', data=df_clean, style='Region', s=100, alpha=0.6)

# Add a reference line to visualize non-linearity
x_max = df_clean['Revenue'].max()
plt.plot([0, x_max], [0, df_clean['Profit'].max()], color='red', linestyle='--', label='Max Observed Profit Line')

plt.title('Figure 6: Transactional Profit vs. Revenue by Region', fontsize=14)
plt.xlabel('Revenue (£)', fontsize=12)
plt.ylabel('Profit (£)', fontsize=12)
plt.gca().ticklabel_format(style='plain', axis='both')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Region', loc='upper left')
plt.grid(axis='both', linestyle='--')
plt.tight_layout()
plt.savefig('Scatter_Profit_Revenue.png')
plt.close()

In [ ]:
# --- 5.2 Interactive Visualisations (Plotly for Dashboard Component) ---

# Aggregate data for the interactive visualization (Revenue by Region and Confectionery)
regional_product_rev = df_clean.groupby(['Region', 'Confectionary'])['Revenue'].sum().reset_index()
regional_product_rev.rename(columns={'Revenue': 'Total Revenue (£)'}, inplace=True)

# Plotly Bar Chart: Total Revenue by Confectionery, allowing color/filter by Region
fig_plotly = px.bar(
    regional_product_rev,
    x='Confectionary',
    y='Total Revenue (£)',
    color='Region',
    title='Interactive Component: Total Revenue by Confectionery (Filterable by Region)',
    labels={'Total Revenue (£)': 'Total Revenue (£)'},
    height=500
)

# Add layout for better large number display and interactivity features
fig_plotly.update_layout(
    yaxis_tickformat='.2s',  # Scientific notation for large numbers
    hovermode="x unified"
)

# Save Plotly figure to a JSON file (required dashboard output)
plotly_json = fig_plotly.to_json()
with open('Interactive_Revenue_Plot.json', 'w') as f:
    f.write(plotly_json)

print("Interactive Plotly JSON file 'Interactive_Revenue_Plot.json' created for dashboard assembly.")

In [ ]:
# --- 2. Dashboard Setup ---

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define options for selectors
all_regions = sorted(df_clean['Region'].unique().tolist())
all_confectioneries = sorted(df_clean['Confectionary'].unique().tolist())
min_year = df_clean['Year'].min()
max_year = df_clean['Year'].max()

# --- 3. Dashboard Layout (6.5 Mockup) ---

sidebar = dbc.Card(
    [
        html.H4("Profit-First Explorer Filters", className="card-title"),
        html.Hr(),

        # Regional Selector (6.3)
        html.Div(
            [
                dbc.Label("Select Region(s)"),
                dcc.Checklist(
                    id='region-selector',
                    options=[{'label': region, 'value': region} for region in all_regions],
                    value=all_regions, # Default to all selected
                    inline=False,
                    className="mb-3",
                    labelStyle={'display': 'block'}
                ),
            ]
        ),

        # Confectionary Type Selector (6.3)
        html.Div(
            [
                dbc.Label("Confectionery Type"),
                dcc.Dropdown(
                    id='confectionery-selector',
                    options=[{'label': 'All Types', 'value': 'All Types'}] +
                            [{'label': type, 'value': type} for type in all_confectioneries],
                    value='All Types',
                    clearable=False,
                    className="mb-3"
                ),
            ]
        ),

        # Time Range Slider (6.3)
        html.Div(
            [
                dbc.Label("Time Range Slider (Year)"),
                dcc.RangeSlider(
                    id='year-slider',
                    min=min_year,
                    max=max_year,
                    value=[min_year, max_year],
                    marks={str(year): str(year) for year in range(min_year, max_year + 1, 1)},
                    step=1
                ),
                html.Br()
            ]
        )
    ],
    body=True,
    className="h-100",
)

content = dbc.Container([
    html.H1("BCC Profitability Dashboard: The Distribution Strategy", className="my-4 text-center"),

    # KPI Cards Row
    dbc.Row([
        dbc.Col(dbc.Card(html.H2(id='kpi-revenue', className="text-center"), body=True, color="primary", inverse=True), md=6),
        dbc.Col(dbc.Card(html.H2(id='kpi-margin', className="text-center"), body=True, color="success", inverse=True), md=6),
    ], className="mb-4"),

    # Profit vs. Revenue Graph (6.3)
    dbc.Row([
        dbc.Col(dcc.Graph(id='profit-revenue-graph'), md=12)
    ]),

], fluid=True)


app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, md=3, className="p-3"),
                dbc.Col(content, md=9, className="p-3"),
            ],
            className="h-100"
        )
    ],
    fluid=True,
    className="vh-100"
)

# --- 4. Interactivity (Callbacks: 6.4) ---

@app.callback(
    [Output('profit-revenue-graph', 'figure'),
     Output('kpi-revenue', 'children'),
     Output('kpi-margin', 'children')],
    [Input('region-selector', 'value'),
     Input('confectionery-selector', 'value'),
     Input('year-slider', 'value')]
)
def update_dashboard(selected_regions, selected_confectionery, year_range):
    # 1. Filter Data based on user inputs
    dff = df_clean[df_clean['Region'].isin(selected_regions)]

    if selected_confectionery != 'All Types':
        dff = dff[dff['Confectionary'] == selected_confectionery]

    dff = dff[(dff['Year'] >= year_range[0]) & (dff['Year'] <= year_range[1])]

    # 2. KPI Calculation
    total_revenue = dff['Revenue'].sum()
    total_profit = dff['Profit'].sum()
    average_margin = (total_profit / total_revenue) if total_revenue > 0 else 0

    # 3. Graph Generation (Profit vs. Revenue Scatter Plot)
    fig = px.scatter(
        dff,
        x='Revenue',
        y='Profit',
        color='Region',
        size='Units Sold', # Visualizing the units sold through size
        hover_data=['Confectionary', 'Date'],
        title="Profit vs. Revenue by Region and Confectionary Type (Filtered Data)",
        labels={'Revenue': 'Total Revenue (£)', 'Profit': 'Total Profit (£)'}
    )

    # Add a visual reference for overall Profit Margin (as a line from origin)
    if total_revenue > 0:
        fig.add_shape(
            type='line',
            x0=0, y0=0,
            x1=total_revenue, y1=total_profit,
            line=dict(color="Red", width=2, dash="dash"),
            name="Avg Margin Line"
        )

    fig.update_layout(
        xaxis_tickformat='.2s',
        yaxis_tickformat='.2s',
        transition_duration=500
    )

    # 4. Return KPIs
    kpi_rev = f"Total Revenue: £{total_revenue:,.0f}"
    kpi_mar = f"Avg Margin: {average_margin:.2%}"

    return fig, kpi_rev, kpi_mar


if __name__ == '__main__':
    print(f"Starting Dash app. Go to http://127.0.0.1:8050/")
    # Debug=True is used here for demonstration/development purposes
    app.run(debug=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as ticker
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
df = pd.read_excel('/content/Confectionary [4564] (1).xlsx')

In [ ]:
print(df.head())

In [ ]:
# Data Cleaning and Transformation
new_columns = {
    'Country(UK)': 'Region',
    'Cost(£)': 'Cost',
    'Profit(£)': 'Profit',
    'Revenue(£)': 'Revenue'
}
df.rename(columns=new_columns, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Confectionary'] = df['Confectionary'].replace('Choclate Chunk', 'Chocolate Chunk')

print("\n--- Data Info After Datetime Conversion and Renaming ---")
print(df.info())

In [ ]:
# Handling Missing Values (Imputation)
print("\n--- Missing Value Count (Before Imputation) ---")
print(df.isnull().sum())


missing_cost_mask = df['Cost'].isnull() & df['Revenue'].notnull() & df['Profit'].notnull()
df.loc[missing_cost_mask, 'Cost'] = df.loc[missing_cost_mask, 'Revenue'] - df.loc[missing_cost_mask, 'Profit']


missing_profit_mask = df['Profit'].isnull() & df['Revenue'].notnull() & df['Cost'].notnull()
df.loc[missing_profit_mask, 'Profit'] = df.loc[missing_profit_mask, 'Revenue'] - df.loc[missing_profit_mask, 'Cost']

print("\n--- Missing Value Count (After Imputation Logic) ---")
print(df.isnull().sum())

In [ ]:
# Data Quality Assessment (Final Cleanup)
df_clean = df[(df['Profit'] > 0) & (df['Revenue'] > 0)].copy()

df_clean.dropna(subset=['Units Sold'], inplace=True)

print("\n--- Final Cleaned Data Info ---")
print(df_clean.info())
print("\n--- Final Cleaned Data Sample ---")
print(df_clean.head())

In [ ]:
# --- 3.1 Regional Distribution of Sales and Profit ---
regional_summary = df_clean.groupby('Region')[['Revenue', 'Profit']].sum().reset_index()
regional_summary['Profit Margin (%)'] = (regional_summary['Profit'] / regional_summary['Revenue']) * 100
regional_summary.sort_values(by='Revenue', ascending=False, inplace=True)

print("\n--- 3.1 Regional Sales and Profit Summary ---")

regional_summary.to_csv("Regional_Summary.csv", index=False)

# Plotting Regional Revenue and Profit
fig, ax = plt.subplots(figsize=(10, 6))
regional_summary_melt = regional_summary.melt(id_vars='Region', value_vars=['Revenue', 'Profit'],
                                              var_name='Metric', value_name='Value (£)')
sns.barplot(x='Region', y='Value (£)', hue='Metric', data=regional_summary_melt, ax=ax, palette={'Revenue': 'cornflowerblue', 'Profit': 'darkorange'})
ax.set_title('Figure 1: Total Revenue and Profit by UK Region', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Region', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Regional_Sales_Profit.png')
plt.close()
regional_summary

In [ ]:
# --- 3.2 Comparative Overview of Confectionary Types ---
confectionary_summary = df_clean.groupby('Confectionary')[['Revenue', 'Profit']].sum().reset_index()
confectionary_summary['Profit Margin (%)'] = (confectionary_summary['Profit'] / confectionary_summary['Revenue']) * 100
confectionary_summary.sort_values(by='Profit Margin (%)', ascending=False, inplace=True)

print("\n--- 3.2 Confectionary Type Sales and Profit Summary (Sorted by Margin) ---")
confectionary_summary
confectionary_summary.to_csv("Confectionary_Summary.csv", index=False)

# Plotting Confectionary Revenue and Profit (sorted by Revenue for visual clarity)
fig, ax = plt.subplots(figsize=(10, 6))
confectionary_summary_melt = confectionary_summary.melt(id_vars='Confectionary', value_vars=['Revenue', 'Profit'],
                                                      var_name='Metric', value_name='Value (£)')
confectionary_order = confectionary_summary.sort_values(by='Revenue', ascending=False)['Confectionary'].tolist()

sns.barplot(x='Confectionary', y='Value (£)', hue='Metric', data=confectionary_summary_melt, ax=ax, palette={'Revenue': 'seagreen', 'Profit': 'goldenrod'}, order=confectionary_order)
ax.set_title('Figure 2: Total Revenue and Profit by Confectionery Type', fontsize=14)
ax.set_ylabel('Value (£)', fontsize=12)
ax.set_xlabel('Confectionery Type', fontsize=12)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Confectionary_Sales_Profit.png')
plt.close()
confectionary_summary

In [ ]:
# --- 3.3 Time Series Analysis ---
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month

# Regional Peak Sales Analysis
regional_yearly_sales = df_clean.groupby(['Year', 'Region'])['Revenue'].sum().reset_index()
peak_sale = regional_yearly_sales.loc[regional_yearly_sales['Revenue'].idxmax()]
time_series_summary = df_clean.groupby('Year')['Revenue'].sum().reset_index()

print("\n--- 3.3 Peak Regional Sales (Region with the single highest yearly revenue) ---")
print(peak_sale)

# Plotting Regional Yearly Revenue to show trends and peak
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Revenue', hue='Region', data=regional_yearly_sales, marker='o')
plt.title('Figure 3: Yearly Revenue Trend by Region (2000-2005)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1000000:.0f}M'))
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('Regional_Yearly_Revenue.png')
plt.close()
time_series_summary

In [ ]:
# --- 3.4 Correlation Analysis between Sales, Revenue, and Profit Margin ---
df_clean['Profit_Margin'] = df_clean['Profit'] / df_clean['Revenue']
correlation_data = df_clean[['Units Sold', 'Revenue', 'Profit', 'Profit_Margin']]
correlation_matrix = correlation_data.corr()

print("\n--- 3.4 Correlation Matrix ---")
print(correlation_matrix)
correlation_matrix.to_csv("Correlation_Matrix.csv")

# Plotting the Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".2f",
            linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Figure 4: Correlation Analysis of Key Financial Metrics', fontsize=14)
plt.tight_layout()
plt.savefig('Correlation_Heatmap.png')
plt.close()

In [ ]:
# 1. Aggregate Revenue, Profit, and calculate Margin by Region and Confectionery
regional_product_summary = df_clean.groupby(['Region', 'Confectionary'])[['Revenue', 'Profit']].sum().reset_index()
regional_product_summary['Profit Margin (%)'] = (regional_product_summary['Profit'] / regional_product_summary['Revenue']) * 100

print("\n--- 4.1 Confectionery Performance Metrics (by Region) ---")
print(regional_product_summary)
regional_product_summary.to_csv("Regional_Product_Metrics.csv", index=False)

In [ ]:
# 2. Find the index corresponding to the maximum Profit Margin for each Region
idx_max = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmax()
largest_margins = regional_product_summary.loc[idx_max].sort_values(by='Profit Margin (%)', ascending=False).reset_index(drop=True)

print("\n--- 4.2 Identification of Confectionery with Largest Profit Margins (by Region) ---")
print(largest_margins)
largest_margins.to_csv("Largest_Profit_Margins_by_Region.csv", index=False)

In [ ]:
# 3. Find the index corresponding to the minimum Profit Margin for each Region
idx_min = regional_product_summary.groupby('Region')['Profit Margin (%)'].idxmin()
smallest_margins = regional_product_summary.loc[idx_min].sort_values(by='Profit Margin (%)', ascending=True).reset_index(drop=True)

print("\n--- 4.3 Identification of Confectionery with Smallest Profit Margins (by Region) ---")
smallest_margins

In [ ]:
# Figure 5: Boxplot of Units Sold Distribution by Confectionery Type
# Shows volatility and average order size (Logistics Insight)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Confectionary', y='Units Sold', data=df_clean, palette='pastel')
plt.title('Figure 5: Distribution of Units Sold by Confectionery Type', fontsize=14)
plt.xlabel('Confectionery Type', fontsize=12)
plt.ylabel('Units Sold (Log Scale)', fontsize=12)
plt.yscale('log') # Use log scale for better visibility of outliers and distribution
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('Boxplot_Units_Sold.png')
plt.close()

In [ ]:
# Figure 6: Scatter Plot of Profit vs. Revenue by Region
# Shows the Volume-Margin Trade-Off (Pricing Insight)
plt.figure(figsize=(12, 7))
sns.scatterplot(x='Revenue', y='Profit', hue='Region', data=df_clean, style='Region', s=100, alpha=0.6)

# Add a reference line to visualize non-linearity
x_max = df_clean['Revenue'].max()
plt.plot([0, x_max], [0, df_clean['Profit'].max()], color='red', linestyle='--', label='Max Observed Profit Line')

plt.title('Figure 6: Transactional Profit vs. Revenue by Region', fontsize=14)
plt.xlabel('Revenue (£)', fontsize=12)
plt.ylabel('Profit (£)', fontsize=12)
plt.gca().ticklabel_format(style='plain', axis='both')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Region', loc='upper left')
plt.grid(axis='both', linestyle='--')
plt.tight_layout()
plt.savefig('Scatter_Profit_Revenue.png')
plt.close()

In [ ]:
# --- 5.2 Interactive Visualisations (Plotly for Dashboard Component) ---

# Aggregate data for the interactive visualization (Revenue by Region and Confectionery)
regional_product_rev = df_clean.groupby(['Region', 'Confectionary'])['Revenue'].sum().reset_index()
regional_product_rev.rename(columns={'Revenue': 'Total Revenue (£)'}, inplace=True)

# Plotly Bar Chart: Total Revenue by Confectionery, allowing color/filter by Region
fig_plotly = px.bar(
    regional_product_rev,
    x='Confectionary',
    y='Total Revenue (£)',
    color='Region',
    title='Interactive Component: Total Revenue by Confectionery (Filterable by Region)',
    labels={'Total Revenue (£)': 'Total Revenue (£)'},
    height=500
)

# Add layout for better large number display and interactivity features
fig_plotly.update_layout(
    yaxis_tickformat='.2s',  # Scientific notation for large numbers
    hovermode="x unified"
)

# Save Plotly figure to a JSON file (required dashboard output)
plotly_json = fig_plotly.to_json()
with open('Interactive_Revenue_Plot.json', 'w') as f:
    f.write(plotly_json)

print("Interactive Plotly JSON file 'Interactive_Revenue_Plot.json' created for dashboard assembly.")

In [ ]:
# --- 2. Dashboard Setup ---

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define options for selectors
all_regions = sorted(df_clean['Region'].unique().tolist())
all_confectioneries = sorted(df_clean['Confectionary'].unique().tolist())
min_year = df_clean['Year'].min()
max_year = df_clean['Year'].max()

# --- 3. Dashboard Layout (6.5 Mockup) ---

sidebar = dbc.Card(
    [
        html.H4("Profit-First Explorer Filters", className="card-title"),
        html.Hr(),

        # Regional Selector (6.3)
        html.Div(
            [
                dbc.Label("Select Region(s)"),
                dcc.Checklist(
                    id='region-selector',
                    options=[{'label': region, 'value': region} for region in all_regions],
                    value=all_regions, # Default to all selected
                    inline=False,
                    className="mb-3",
                    labelStyle={'display': 'block'}
                ),
            ]
        ),

        # Confectionary Type Selector (6.3)
        html.Div(
            [
                dbc.Label("Confectionery Type"),
                dcc.Dropdown(
                    id='confectionery-selector',
                    options=[{'label': 'All Types', 'value': 'All Types'}] +
                            [{'label': type, 'value': type} for type in all_confectioneries],
                    value='All Types',
                    clearable=False,
                    className="mb-3"
                ),
            ]
        ),

        # Time Range Slider (6.3)
        html.Div(
            [
                dbc.Label("Time Range Slider (Year)"),
                dcc.RangeSlider(
                    id='year-slider',
                    min=min_year,
                    max=max_year,
                    value=[min_year, max_year],
                    marks={str(year): str(year) for year in range(min_year, max_year + 1, 1)},
                    step=1
                ),
                html.Br()
            ]
        )
    ],
    body=True,
    className="h-100",
)

content = dbc.Container([
    html.H1("BCC Profitability Dashboard: The Distribution Strategy", className="my-4 text-center"),

    # KPI Cards Row
    dbc.Row([
        dbc.Col(dbc.Card(html.H2(id='kpi-revenue', className="text-center"), body=True, color="primary", inverse=True), md=6),
        dbc.Col(dbc.Card(html.H2(id='kpi-margin', className="text-center"), body=True, color="success", inverse=True), md=6),
    ], className="mb-4"),

    # Profit vs. Revenue Graph (6.3)
    dbc.Row([
        dbc.Col(dcc.Graph(id='profit-revenue-graph'), md=12)
    ]),

], fluid=True)


app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, md=3, className="p-3"),
                dbc.Col(content, md=9, className="p-3"),
            ],
            className="h-100"
        )
    ],
    fluid=True,
    className="vh-100"
)

# --- 4. Interactivity (Callbacks: 6.4) ---

@app.callback(
    [Output('profit-revenue-graph', 'figure'),
     Output('kpi-revenue', 'children'),
     Output('kpi-margin', 'children')],
    [Input('region-selector', 'value'),
     Input('confectionery-selector', 'value'),
     Input('year-slider', 'value')]
)
def update_dashboard(selected_regions, selected_confectionery, year_range):
    # 1. Filter Data based on user inputs
    dff = df_clean[df_clean['Region'].isin(selected_regions)]

    if selected_confectionery != 'All Types':
        dff = dff[dff['Confectionary'] == selected_confectionery]

    dff = dff[(dff['Year'] >= year_range[0]) & (dff['Year'] <= year_range[1])]

    # 2. KPI Calculation
    total_revenue = dff['Revenue'].sum()
    total_profit = dff['Profit'].sum()
    average_margin = (total_profit / total_revenue) if total_revenue > 0 else 0

    # 3. Graph Generation (Profit vs. Revenue Scatter Plot)
    fig = px.scatter(
        dff,
        x='Revenue',
        y='Profit',
        color='Region',
        size='Units Sold', # Visualizing the units sold through size
        hover_data=['Confectionary', 'Date'],
        title="Profit vs. Revenue by Region and Confectionary Type (Filtered Data)",
        labels={'Revenue': 'Total Revenue (£)', 'Profit': 'Total Profit (£)'}
    )

    # Add a visual reference for overall Profit Margin (as a line from origin)
    if total_revenue > 0:
        fig.add_shape(
            type='line',
            x0=0, y0=0,
            x1=total_revenue, y1=total_profit,
            line=dict(color="Red", width=2, dash="dash"),
            name="Avg Margin Line"
        )

    fig.update_layout(
        xaxis_tickformat='.2s',
        yaxis_tickformat='.2s',
        transition_duration=500
    )

    # 4. Return KPIs
    kpi_rev = f"Total Revenue: £{total_revenue:,.0f}"
    kpi_mar = f"Avg Margin: {average_margin:.2%}"

    return fig, kpi_rev, kpi_mar


if __name__ == '__main__':
    print(f"Starting Dash app. Go to http://127.0.0.1:8050/")
    # Debug=True is used here for demonstration/development purposes
    app.run(debug=True)